In [ ]:
# default_exp exec.train_tacotron2

In [ ]:
# export
from uberduck_ml_dev.trainer.tacotron2 import Tacotron2Trainer
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.models.tacotron2 import DEFAULTS as TACOTRON2_DEFAULTS
import argparse
import sys
import json
import torch
from torch import multiprocessing as mp

In [ ]:
# export
def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config")
    args = parser.parse_args(args)
    return args


def run(rank, device_count, hparams):
    trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
    try:
        trainer.train()
    except Exception as e:
        print(f"Exception raised while training: {e}")
        # TODO: save state.
        raise e

In [ ]:
# export
try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    config = TACOTRON2_DEFAULTS.values()
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))
    config.update(vars(args))
    hparams = HParams(**config)
    if hparams.distributed_run:
        device_count = torch.cuda.device_count()
        mp.spawn(run, (device_count, hparams), device_count)
    else:
        run(None, None, hparams)

In [ ]:
# skip
config = TACOTRON2_DEFAULTS.values()
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
hparams = HParams(**config)
print(hparams)
if hparams.distributed_run:
    device_count = torch.cuda.device_count()
    mp.spawn(run, (device_count, hparams), device_count)
else:
    run(None, None, hparams)

[('attention_dim', 128), ('attention_location_kernel_size', 31), ('attention_location_n_filters', 32), ('attention_rnn_dim', 1024), ('batch_size', 16), ('checkpoint_name', None), ('checkpoint_path', 'test/fixtures/results/checkpoints'), ('coarse_n_frames_per_step', None), ('cudnn_enabled', True), ('dataset_path', './dataset'), ('debug', False), ('decay_rate', 8000), ('decay_start', 15000), ('decoder_rnn_dim', 1024), ('distributed_run', False), ('encoder_embedding_dim', 512), ('encoder_kernel_size', 5), ('encoder_n_convolutions', 3), ('epochs', 5), ('epochs_per_checkpoint', 4), ('filter_length', 1024), ('fp16_run', False), ('gate_threshold', 0.5), ('grad_clip_thresh', 1.0), ('hop_length', 256), ('ignore_layers', ['speaker_embedding.weight']), ('include_f0', False), ('learning_rate', 0.001), ('log_dir', 'test/fixtures/results/logs'), ('mask_padding', True), ('max_decoder_steps', 1000), ('max_wav_value', 32768.0), ('mel_fmax', 8000), ('mel_fmin', 0), ('n_frames_per_step_initial', 1), ('n_

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/uberduck_ml_dev/trainer/tacotron2.py:444: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  model.parameters(), self.grad_clip_thresh


Loss: 1.4807120403274894
logging_time: 0.0081
here
Loss: 16.083421297371387
logging_time: 0.0084
here
Loss: 3.4663464445620775
logging_time: 0.0083
here
Loss: 5.109222259372473
logging_time: 0.0079
start validate 512291.396348757
here
here
here
here
Average loss: 2.684908957220614
here
Loss: 2.713851787149906
logging_time: 0.0081
here
Loss: 1.562770925462246
logging_time: 0.0083
here
Loss: 1.7748498525470495
logging_time: 0.0078
here
Loss: 1.348441630601883
logging_time: 0.0083
start validate 512312.866006687
here
here
here
here
Average loss: 1.4870205072220415
here
Loss: 1.5133823892101645
logging_time: 0.0084
here
Loss: 1.4889013804495335
logging_time: 0.0061
here
Loss: 1.025931190699339
logging_time: 0.0082
here
Loss: 1.0355854467488825
logging_time: 0.0085
start validate 512333.495222449
here
here
here
here
Average loss: 1.0223602829501033
here
Loss: 0.9582625976763666
logging_time: 0.0085
here
Loss: 0.9869085298851132
logging_time: 0.0082
here
Loss: 0.9209097870625556
logging_time